Task 2 - User Engagement analysis

As telecom brands are the data providers of all online activities, meeting user requirements, and creating an engaging user experience is a prerequisite for them. Building & improving the QoS (Quality of Service) to leverage the mobile platforms and to get more users for the business is good but the success of the business would be determined by the user engagement and activity of the customers on available apps. 

In telecommunication, tracking the user activities on the database sessions is a good starting point to appreciate the user engagement for the overall applications and per application as well. If we can determine the level of engagement of a random user for any application, then it could help the technical teams of the business to know where to concentrate network resources for different clusters of customers based on the engagement scores.

In the current dataset you’re expected to track the user’s engagement using the following engagement metrics: 
sessions frequency 

the duration of the session 

the sessions total traffic (download and upload (bytes))

Task 2.1 - Based on the above submit python script and slide :

1.Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric 

2.Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 

3.Compute the minimum, maximum, average & total non- normalized metrics for each cluster. Interpret your results visually with accompanying text explaining your findings.

4.Aggregate user total traffic per application and derive the top 10 most engaged users per application

5.Plot the top 3 most used applications using appropriate charts. 

6.Using k-means clustering algorithm, group users in k engagement clusters based on the engagement metrics: 
What is the optimized value of k (use elbow method for this)?  
Interpret your findings. 



In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_columns', None)

In [2]:
df=pd.read_csv('../data/to_trasform.csv')

In [3]:
df.head(2)

,Unnamed: 0,Bearer_Id,Dur. (ms),IMSI,MSISDN,IMEI,last_location,avg_rtt_dl,avg_rtt_ul,throughput_avg_dl_kpbs,throughput_avg_ul_kpbs,retrans_packets_dl_b,retrans_packets_ul_b,tp_dl_below_50kbps_pc,tp_dl_50_250kbps_pc,tp_dl_250kbps_1mbps_pc,tp_dl_above_1mbps_pc,tp_ul_below_10kpbs_pc,tp_ul_10_50_kbps_pc,tp_ul_50_300_kbps_pc,tp_ul_above_300_kpbs_pc,activity_duration_dl,activity_duration_ul,Dur. (ms).1,phone_company,phone_name,t_vol_dl_above_6250B,t_vol_ul_above_1250B,socials_dl_b,socials_ul_b,google_dl_b,google_ul_b,email_dl_b,email_ul_b,youtube_dl_b,youtube_ul_b,netflix_dl_b,netflix_ul_b,gaming_dl_b,gaming_ul_b,other_dl_b,other_ul_b,Total_ul_b,Total_dl_b,total_ul_data,total_dl_data,socials_total_b,google_total_b,email_total_b,youtube_total_b,netflix_total_b,gaming_total_b,other_total_b,total_b
0,0,13114483460844900352,1823652,208201448079117,33664962239,35521209507511,9.16456699548519E+015,42,5,23,44,20884183,766247,100,0,0,0,100,0,0,0,37624,38787,1823652892,Samsung,Samsung Galaxy A5 Sm-A520F,213,214,1545765,24420,1634479,1271433,3563542,137762,15854611,2501332,8198936,9656251,278082303,14344150,171744450,8814393,36749741,308879636,36749741,480624086,1570185,2905912,3701304,18355943,17855187,292426453,180558843,345629377
1,1,13114483482878900224,1365104,208201909211140,33681854413,35794009006359,L77566A,65,5,16,26,20884183,766247,100,0,0,0,100,0,0,0,168,3560,1365104371,Samsung,Samsung Galaxy J5 (Sm-J530),971,1022,1926113,7165,3493924,920172,629046,308339,20247395,19111729,18338413,17227132,608750074,1170709,526904238,15055145,53800391,653384965,53800391,1180289203,1933278,4414096,937385,39359124,35565545,609920783,541959383,707185356


In the current dataset you’re expected to track the user’s engagement using the following engagement metrics: 

1.sessions frequency 

2.the duration of the session 

3.the sessions total traffic (download and upload (bytes))



1.Aggregate the above metrics per customer id (MSISDN) and report the top 10 customers per engagement metric 

In [4]:
import sys
sys.path.insert(0,'../scripts')
from results_pickler import ResultPickler

In [5]:
results = ResultPickler()

In [6]:
session_f=df.groupby(['MSISDN'])["Dur. (ms).1"].count().reset_index(name='session_count')

In [7]:
session_frequency=session_f.sort_values(['session_count'], ascending=False).head(10)

In [8]:
results.add_data('session_frequency',session_frequency)

Aggregation by session duration

In [9]:
session_d=df.groupby(['MSISDN'])["Dur. (ms).1"].sum().reset_index(name='total_duration')

In [10]:
session_duration=session_d.sort_values(['total_duration'], ascending=False).head(10)

In [11]:
results.add_data('session_duration',session_duration)

Total upload and download

In [12]:
df['total_dl_ul']=df['total_dl_data'] + df['total_ul_data']

In [13]:
total_s_b=df.groupby(['MSISDN'])["total_dl_ul"].sum().reset_index(name='total_data')

In [14]:
total_session_b=total_s_b.sort_values(['total_data'], ascending=False).head(10)

In [15]:
results.add_data('total_session_b',total_session_b)

# Task 2.1 

 
Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 

Compute the minimum, maximum, average & total non- normalized metrics for each cluster. Interpret your results visually with accompanying text explaining your findings.

.


In [16]:
data  = pd.merge(total_s_b, session_d, on='MSISDN', how='left')
data  = pd.merge(data, session_f, on='MSISDN', how='left')

In [17]:
data.head(10)

,MSISDN,total_data,total_duration,session_count
0,33601001722,1255787564,116720140,1
1,33601001754,436417344,181230963,1
2,33601002511,1091052984,134969374,1
3,33601007832,447568699,49878024,1
4,33601008617,2235054657,37104453,2
5,33601010682,1095575829,253983077,2
6,33601011634,859918491,128360523,2
7,33601011959,510773383,86399977,1
8,33601014694,2016653584,495702147,2
9,33601020306,866204929,124854778,1


# Normalization

Normalization

Scaling just changes the range of your data. Normalization is a more radical transformation. The point of normalization is to change your observations so that they can be described as a normal distribution.

    Normal distribution: Also known as the "bell curve", this is a specific statistical distribution where a roughly equal observations fall above and below the mean, the mean and the median are the same, and there are more observations closer to the mean. The normal distribution is also known as the Gaussian distribution.

In general, you'll normalize your data if you're going to be using a machine learning or statistics technique that assumes your data is normally distributed. Some examples of these include linear discriminant analysis (LDA) and Gaussian naive Bayes. (Pro tip: any method with "Gaussian" in the name probably assumes normality.)

The method you will be using to normalize here is called the Normalizer method from sklearn. Let's take a quick peek at what normalizing some data looks like:

# Normalize each engagement metric and run a k-means (k=3) to classify customers in three groups of engagement. 

In [18]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import decomposition
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

C:\Users\kachase\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [19]:
from sklearn.preprocessing import Normalizer
import seaborn as sns


norm = Normalizer()
    
normalized_data = norm.fit_transform(data)




In [20]:
normalized_data

array([[9.99296320e-01, 3.73472167e-02, 3.47126575e-03, 2.97400753e-11],
       [9.99901122e-01, 1.29869399e-02, 5.39308455e-03, 2.97580748e-11],
       [9.99465184e-01, 3.24534803e-02, 4.01467754e-03, 2.97451001e-11],
       ...,
       [9.99999938e-01, 2.18891917e-04, 2.74438393e-04, 3.12791182e-13],
       [1.00000000e+00, 2.19775015e-06, 7.50831375e-07, 2.96735872e-15],
       [1.00000000e+00, 5.98725178e-07, 9.85774082e-07, 1.13327661e-15]])

k-means

In [21]:
kmeanModel = KMeans(n_clusters=3, random_state = 0)
kmeanModel.fit_predict(normalized_data )


array([0, 0, 0, ..., 0, 0, 0])

In [22]:
model1_centroids=kmeanModel.cluster_centers_

In [23]:
model1_centroids

array([[9.99615529e-01, 2.59326565e-02, 3.21799066e-03, 3.13791393e-11],
       [9.92306726e-01, 1.17472729e-01, 1.56727598e-02, 1.18362870e-10],
       [9.98285422e-01, 5.68149065e-02, 5.43864352e-03, 5.52081640e-11]])

In [24]:
kmeanModel.inertia_

17.05467424862771

In [25]:
kmeanModel.labels_

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
features = data.copy()

In [27]:
labels = kmeanModel.labels_
varieties = features['MSISDN'].to_list()

In [28]:
np.array(np.unique(labels, return_counts=True)).T

array([[    0, 74464],
       [    1,  4728],
       [    2, 27279]], dtype=int64)

In [29]:
data_cluster = pd.DataFrame({'CLUSTERS': labels, 'MSISDN': varieties})

In [30]:
data_cluster['CLUSTERS'].value_counts().to_frame()

,CLUSTERS
0,74464
2,27279
1,4728


In [31]:
data_cluster.head(5)

,CLUSTERS,MSISDN
0,0,33601001722
1,0,33601001754
2,0,33601002511
3,0,33601007832
4,2,33601008617


In [32]:
data2 = pd.merge(data,data_cluster, on='MSISDN')
data2.head()

,MSISDN,total_data,total_duration,session_count,CLUSTERS
0,33601001722,1255787564,116720140,1,0
1,33601001754,436417344,181230963,1,0
2,33601002511,1091052984,134969374,1,0
3,33601007832,447568699,49878024,1,0
4,33601008617,2235054657,37104453,2,2


# Compute the minimum, maximum, average & total non- normalized metrics for each cluster. Interpret your results visually with accompanying text explaining your findings.

In [33]:
data2_0 = data2[data2['CLUSTERS'] == 0]
data2_1 = data2[data2['CLUSTERS'] == 1]
data2_2 = data2[data2['CLUSTERS'] == 2]

In [34]:
data2_0.shape

(74464, 5)

In [35]:
data2_0.describe()

,MSISDN,total_data,total_duration,session_count,CLUSTERS
count,74464,74464,74464,74464,74464
mean,50091236646,873347106,108418414,1,0
std,3461281088331,301650254,80345049,0,0
min,33601001722,58362293,7142988,1,0
25%,33650788515,657241906,57179283,1,0
50%,33663668868,896656394,86583414,1,0
75%,33683645017,1111628602,143708415,1,0
max,882397108489451,1407548816,2083210006,4,0


In [36]:
data2_1.shape

(4728, 5)

In [37]:
data2_1.describe()

,MSISDN,total_data,total_duration,session_count,CLUSTERS
count,4728,4728,4728,4728,4728
mean,33677300944,3992605579,535322107,4,1
std,43504188,1089282447,629948803,1,0
min,33601034530,2608284800,33476692,2,1
25%,33658759616,3249926495,296320128,3,1
50%,33664184722,3686552172,387414358,4,1
75%,33684116055,4398808620,518331394,5,1
max,33789881186,16341524035,18553763486,18,1


In [38]:
data2_2.shape

(27279, 5)

In [39]:
data2_2.describe()

,MSISDN,total_data,total_duration,session_count,CLUSTERS
count,27279,27279,27279,27279,27279
mean,33674129753,1915778448,183343303,2,2
std,45338633,418152505,133949395,1,0
min,33601008617,1356439704,7342167,1,2
25%,33650875060,1548680042,86400009,1,2
50%,33663516947,1816921603,170093196,2,2
75%,33682870024,2216556568,246468769,2,2
max,33789980299,2994737796,1867677937,5,2


In [41]:
pd.options.display.float_format = '{:,.2f}'.format
average_measures = data2.iloc[:,2:].groupby(['CLUSTERS']).mean()


In [42]:
average_measures.T.style.highlight_max(color = 'lightgreen', axis = 1).format("{:,.2f}")

CLUSTERS,0,1,2
total_duration,"108,418,414.31","535,322,107.01","183,343,303.32"
session_count,1.06,4.02,1.86


In [43]:
average_measures.T.style.highlight_min(color = 'pink', axis = 1).format("{:,.2f}")

CLUSTERS,0,1,2
total_duration,"108,418,414.31","535,322,107.01","183,343,303.32"
session_count,1.06,4.02,1.86


# Aggregation based on social media

In [ ]:
socials=df.groupby(['MSISDN'])["socials_total_b"].sum().reset_index(name='socials_total')

In [ ]:
social_media_traffic=socials.sort_values(['socials_total'], ascending=False).head(10)

In [ ]:
results.add_data('social_media_traffic',social_media_traffic)

# Aggregation based on google

In [ ]:
google=df.groupby(['MSISDN'])["google_total_b"].sum().reset_index(name='google_total')

In [ ]:
google_traffic=google.sort_values(['google_total'], ascending=False).head(10)

In [ ]:
results.add_data('google_traffic',google_traffic)

# Aggregation based on email

In [ ]:
email=df.groupby(['MSISDN'])["email_total_b"].sum().reset_index(name='mail_total')

In [ ]:
email_traffic=email.sort_values(['mail_total'], ascending=False).head(10)

In [ ]:
results.add_data('email_traffic',email_traffic)

# Aggregation based on YouTube

In [ ]:
youtube=df.groupby(['MSISDN'])["youtube_total_b"].sum().reset_index(name='yt_total')

In [ ]:
youtube_traffic=youtube.sort_values(['yt_total'], ascending=False).head(10)

In [ ]:
results.add_data('youtube_traffic',youtube_traffic)

# Aggregation based on Netflix

In [ ]:
netflix=df.groupby(['MSISDN'])["netflix_total_b"].sum().reset_index(name='netflix_total')

In [ ]:
netflix_traffic=netflix.sort_values(['netflix_total'], ascending=False).head(10)

In [ ]:
results.add_data('netflix_traffic',netflix_traffic)

# Aggregation based on Gaming

In [ ]:
gaming=df.groupby(['MSISDN'])["gaming_total_b"].sum().reset_index(name='gaming_total')

In [ ]:
gaming_traffic=gaming.sort_values(['gaming_total'], ascending=False).head(10)

In [ ]:
results.add_data('gaming_traffic',gaming_traffic)

# Aggregation based on Other

In [ ]:
other=df.groupby(['MSISDN'])["other_total_b"].sum().reset_index(name='other_total')

In [ ]:
other_traffic=other.sort_values(['other_total'], ascending=False).head(10)

In [ ]:
results.add_data('other_traffic',other_traffic)

# App perfomance

In [ ]:
most_used_df = df.copy(deep=True)
most_used_df = most_used_df.loc[:,[ 'socials_total_b','google_total_b','email_total_b','youtube_total_b','netflix_total_b','gaming_total_b','other_total_b']]

In [ ]:
total_app_data_usage = pd.DataFrame(most_used_df.sum(), columns=['Total Data (Bytes)'])
most_used_apps = total_app_data_usage.sort_values(by='Total Data (Bytes)',ascending=False).iloc[:5,:]

#Saving the data
results.add_data('most_used_apps',most_used_apps)
#result
most_used_apps

Using k-means clustering algorithm, group users in k engagement clusters based on the engagement metrics: What is the optimized value of k (use elbow method for this)?
Interpret your findings

In [ ]:
data  = pd.merge(total_s_b, session_d, on='MSISDN', how='left')
data  = pd.merge(data, session_f, on='MSISDN', how='left')

In [ ]:
engagement=pd.merge(data, other, on='MSISDN', how='left')
engagement=pd.merge(engagement, gaming, on='MSISDN', how='left')
engagement=pd.merge(engagement, netflix, on='MSISDN', how='left')
engagement=pd.merge(engagement, youtube, on='MSISDN', how='left')
engagement=pd.merge(engagement, google, on='MSISDN', how='left')
engagement=pd.merge(engagement, email, on='MSISDN', how='left')
#engagement=pd.merge(engagement, other_traffic, on='MSISDN', how='left')
engagement=pd.merge(engagement, socials, on='MSISDN', how='left')

In [ ]:
engagement.head(2)

In [ ]:
cat_features=['MSISDN']

In [ ]:
numerical_features = ['total_data','total_duration','session_count','other_total','gaming_total','netflix_total','yt_total','google_total','mail_total','socials_total']

In [ ]:
input_features = numerical_features + cat_features

In [ ]:
features = engagement.copy()

# Encoding categorical variables

In [ ]:
#casting the columns into strings
labelencoder = LabelEncoder()

#features[cat_features] = features[cat_features].apply(LabelEncoder().fit_transform)
features[cat_features] = features[cat_features].apply(lambda col: LabelEncoder().fit_transform(col.astype(str)), axis=0, result_type='expand')

# Scaling Numerical Data

In [ ]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(features[input_features])
#x_scaled

In [ ]:
import scipy.stats as stats
X_scaled = stats.zscore(x_scaled)
#X_scaled

# Elbow method to determine the clusters

In [ ]:
X = X_scaled


#sns.set(rc={"figure.figsize": (6, 8)})
plt.figure(figsize=(10,6))
cs = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    cs.append(kmeans.inertia_)
plt.plot(range(1, 10), cs, '-o')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('CS')
plt.show()

In [ ]:
from kneed import KneeLocator
kl = KneeLocator(range(1, 10), cs, curve="convex", direction="decreasing")
kl.elbow

In [ ]:
engagement_model = KMeans(n_clusters=3, random_state = 0)
engagement_model.fit_predict(x_scaled)
#print(kmeans.cluster_centers_)

In [ ]:
engagement_model.inertia_

In [ ]:
engagement_model.labels_
#print(kmeans.labels_)

In [ ]:
engagement_centroids=engagement_model.cluster_centers_

In [ ]:
print(engagement_centroids)

In [ ]:
features = engagement.copy()

In [ ]:
labels = engagement_model.labels_
varieties = features['MSISDN'].to_list()

In [ ]:
np.array(np.unique(labels, return_counts=True)).T

In [ ]:
engagement_cluster = pd.DataFrame({'CLUSTERS': labels, 'MSISDN': varieties})

In [ ]:
engagement_cluster['CLUSTERS'].value_counts().to_frame()

In [ ]:
engagement_cluster.head()

In [ ]:
engagement2 = pd.merge(engagement,engagement_cluster, on='MSISDN')
engagement2.shape

In [ ]:

engagement2.head(3)

enga

In [ ]:
engagement_cluster_0 = engagement2[engagement2['CLUSTERS'] == 0]
engagement_cluster_1 = engagement2[engagement2['CLUSTERS'] == 1]
engagement_cluster_2 = engagement2[engagement2['CLUSTERS'] == 2]

In [ ]:
engagement_cluster_0.describe()

In [ ]:
engagement_cluster_1.describe()

In [ ]:
engagement_cluster_2.describe()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
average_measures = engagement2.iloc[:,2:].groupby(['CLUSTERS']).mean()

In [ ]:
average_measures.T.style.highlight_max(color = 'lightgreen', axis = 1).format("{:,.2f}")

In [ ]:
average_measures.T.style.highlight_min(color = 'pink', axis = 1).format("{:,.2f}")

In [ ]:
#saving our results to a pickle file
results.save_data('../data/engagement_results.pickle')

In [ ]:
normalized_data

In [ ]:
engagement_normalized = pd.DataFrame(normalized_data, columns=['MSISDN','total_data','total_duration','session_count'])

In [ ]:
engagement_normalized .head()

In [ ]:
model1_centroids

In [ ]:
centroids_df = pd.DataFrame(model1_centroids, columns=['MSISDN','total_data','total_duration','session_count'])


In [ ]:
centroids_df.head(2)

In [ ]:
centroids_df.to_csv('../data/engagement_centroid.csv')

In [ ]:
engagement_normalized.to_csv('../data/engagement_cluster.csv')